# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map=city_data_df.hvplot.points('Lat','Lng',geo=True,size='Humidity',color='Humidity',tiles="OSM")

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities=city_data_df[(city_data_df["Max Temp"]<26)&(city_data_df["Max Temp"]>20)&(city_data_df["Cloudiness"]==0)&(city_data_df["Humidity"]<60)]
# Drop any rows with null values
ideal_cities=ideal_cities.dropna(how='any')

# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,velingara,-19.3313,36.8201,24.53,53,0,5.62,MZ,1688063644
50,50,stanley,47.6595,-102.7302,25.23,41,0,3.69,US,1688063707
63,63,bor,39.4842,115.4140,22.67,34,0,2.70,CN,1688063745
86,86,ishim,34.1226,47.9609,20.54,18,0,1.86,IR,1688063804
124,124,nusaybin,27.7221,62.1361,25.21,15,0,1.95,IR,1688063909


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_cities[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
28,velingara,MZ,-19.3313,36.8201,53,
50,stanley,US,47.6595,-102.7302,41,
63,bor,CN,39.4842,115.4140,34,
86,ishim,IR,34.1226,47.9609,18,
124,nusaybin,IR,27.7221,62.1361,15,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accomodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=row[2]
    lng=row[3]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"cicle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
velingara - nearest hotel: No hotel found
stanley - nearest hotel: No hotel found
bor - nearest hotel: No hotel found
ishim - nearest hotel: No hotel found
nusaybin - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
28,velingara,MZ,-19.3313,36.8201,53,No hotel found
50,stanley,US,47.6595,-102.7302,41,No hotel found
63,bor,CN,39.4842,115.4140,34,No hotel found
86,ishim,IR,34.1226,47.9609,18,No hotel found
124,nusaybin,IR,27.7221,62.1361,15,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map=hotel_df.hvplot.points('Lat','Lng',geo=True,size='Humidity',color='Humidity',tiles='OSM',hover=True)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (Humidity)